In [1]:
from glob import glob 
import pandas as pd
import os
from utils import load_data_to_dataframe 
import sys 
sys.path.append('/home/work/yuna/VLMEval/analysis/utils')

### Load old vqa quesitons 
dfs = []
for file in glob("./questions/VQA*"): 
    df = load_data_to_dataframe(file)
    dfs.append(df)

### ALL MMStar questions 
mmstar = "/home/work/yuna/HPA/eda/questions/raw/ACL_mmstar_questions.xlsx"
pd.ExcelFile(mmstar).sheet_names 

['s1', 'others']

In [2]:
from question_type_mapper import question_type 

print('Total # of unique VQA question types:', len(set(question_type.values())) )# total unique question type  

### all VQA validation questions 
md = pd.read_csv('/home/work/yuna/VLMEval/results/vqav2_224_InternVL3-8B-blind.csv')
md['question_type'] = md['question_id'].map(question_type)
print(len(md.question_type.unique()) )
model_qids= md.question_id.unique()  # all the validation questions id 
len(md), len(model_qids) 

Total # of unique VQA question types: 65
64


(1000, 1000)

In [39]:
# Chosen MMStar old questions 
df = pd.read_excel(mmstar, header=None, sheet_name='s1') 
df.columns = ['category', 'l2_category', 'question_id', 'question_en', 'question_kr', '', '', '', '', '', '']
df[['question_en', 'options']] = df['question_en'].str.split('Options:', n=1, expand=True)
df = df.dropna(axis=0) 
df['answer_type'] = 'choice' 
df = df.drop(columns=['']) # .to_csv('./questions/mmstar_v1_choice.csv')
len(df.columns), sorted(df.columns)
df['question_id'] = df['question_id'].astype(int)
s1 = df['question_id'].unique()

In [54]:
# Chosen MMStar old questions 
df = pd.read_excel(mmstar, sheet_name='others') 
df.columns = ['question_id' ,'category', 'l2_category', 'question', 'answer', 'score']
df[['question_en', 'options']] = df['question'].str.split('Options:', n=1, expand=True)
df = df.dropna(axis=0) 
df['answer_type'] = 'choice' 
df['question_id'] = df['question_id'].astype(int) 
# df = df.drop(columns=['']) # .to_csv('./questions/mmstar_v1_choice.csv')
len(df.columns), sorted(df.columns)

(9,
 ['answer',
  'answer_type',
  'category',
  'l2_category',
  'options',
  'question',
  'question_en',
  'question_id',
  'score'])

In [128]:
vqa = pd.read_json("/home/work/yuna/HPA/data/vqav2_1k/val.json", lines=False)

In [ ]:
mm = sample_new_qs(df[~df['question_id'].isin(s1_2)], num_sample=100, types=['category', 'l2_category'], answer_type='choice', preprocess=True, filename=None) 

In [25]:
len(df)

992

In [7]:
from openai import OpenAI
import os 
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY) # os.getenv("OPENAI_API_KEY"))

def translate(question, options):

    prompt = f"""
            You are a precise translation and data-formatting assistant.

            Task:
            Given a multiple-choice question and its options in English, 
            return the result as a strict JSON dictionary with the following fields:
            - question_kr: Translate the QUESTION into Korean 
            - options: Insert translated korean options after its English choices separated by a blank space in the same string format. Do not remove the original English options. Numbers don't need to be translated in both languages.

            Input:
            QUESTION: {question}
            OPTIONS_STRING: {options}
            """

    response = client.chat.completions.create(
        model="gpt-5-nano",   # or your preferred model
        messages=[{"role": "user", "content": prompt}],
        temperature=1,
    )

    output = response.choices[0].message.content.strip()
    return output

In [58]:
from sklearn.model_selection import train_test_split 

# Sample new questions 
def sample_new_qs(df, num_sample=0, types=['category', 'l2_category'], answer_type='choice', preprocess=True, filename=None):  

    # preprocessing 
    df = df.rename(columns={"Unnamed: 0": 'qid', 'question_id': 'qid', 'question': 'question_en'}).dropna(axis=0) 
    df['qid'] = df['qid'].astype('Int64') 
    df['answer_type'] = answer_type 

    ## split options from questions 
    if answer_type == 'choice' and 'options' not in df.columns:     
        df[['question_en', 'options']] = df['question_en'].str.split('Options:', n=1, expand=True)

    if num_sample > 0 : ### sample new quetsions 
        print(f"sampling new #{num_sample} from {types}")

        df, _ = train_test_split(
            df,
            train_size=num_sample, 
            stratify=df[types], 
            random_state=42
        )  

    if preprocess: 

        df['question_kr'] = None 

        for idx, row in df.iterrows():
            question = row["question_en"]
            options = row["options"]
            output = translate(question, options) 
            print(output)

            try:
                parsed = json.loads(output)
            except json.JSONDecodeError:
                print("JSON decode error:", output)
                parsed = None

            df.at[idx, 'question_kr'] = parsed['question_kr']
            df.at[idx, 'options'] = parsed['options']

    if filename is not None: 
        df.to_csv(f'./questions/processed/{filename}_{answer_type}.csv')
        print(f'saved to ./questions/processed/{filename}_{answer_type}.csv') 
    
    # print(len(v2.columns),sorted( v2.columns))

    return df 


In [140]:
import pandas as pd 
from glob import glob 

dfs=[]
for qs in glob("/home/work/yuna/HPA/eda/questions/processed/*.csv"):
    df = pd.read_csv(qs).rename(columns={'question_id':  "qid"}).drop_duplicates(subset=['qid']) 
    # print(df.columns)
    dfs.append(df) 
df = pd.concat(dfs)
df[['qid', 'category', 'l2_category', 'question_en', 'options','answer_type', 'question_kr', 'index','question_type']]
print(len(df[~(df['category'] == 'science & technology')]))
df = df[~(df['category'] == 'science & technology')]
df.to_csv('./full_qs_s1.csv')

526


In [167]:
dist = []
for f,v in fd.items() : 
    if f not in sd.keys(): 
        sd[f] = 0
    s = sd[f]
    # print(f, s/v)
    dist.append({
        '#subset': s, 
        '#total': v, 
        'done': s/v, 
        'question_type': f
    })

In [176]:
vqa_qids=df.dropna(subset=['question_type']).qid.unique()
len(vqa_qids)

277

In [180]:
vqa_unsampled = vqa[~vqa['question_id'].isin(vqa_qids)]

In [183]:
d = pd.DataFrame(dist).sort_values(by=['done', '#total'], ascending=False)
qtypes_to_sample = d[d['done'] ==0].question_type.values

In [ ]:
#get distributions of question type data from vqa 
sd = df.groupby(['question_type'])['qid'].count().reset_index().set_index('question_type')['qid'].to_dict() # subset 
fd = vqa.groupby(['question_type']).count().reset_index().set_index('question_type')['question_id'].to_dict() #fullset 

In [133]:
question_distribution = vqa['question_type'].value_counts()
question_distribution

question_type
is the               88
how many             85
what                 84
what color is the    59
what is the          52
                     ..
has                   2
was                   2
what room is          2
is this person        1
is the person         1
Name: count, Length: 64, dtype: int64

In [58]:
mm = pd.read_csv('./questions/processed/mmstar_translated_s1_2.csv').dropna(axis=1)
# mm[columns].to_csv('./questions/processed/mmstar_translated_s1_2.csv')
# mm[columns].to_csv('./questions/processed/mmstar_translated_s1_2.csv', index=False)

In [87]:
columns = ['qid',	'category',	'l2_category','question_en',	'options',	'answer_type',	'question_kr'] 
mm = pd.read_csv('./questions/processed/mmstar_translated_s3.csv') # .rename(columns={'question_en.1': 'question_en'})[columns]
# mm.to_csv('/home/work/yuna/HPA/eda/questions/processed/mmstar_translated_s3.csv', index=False) 
# mm = mm.drop(columns=['question_en'])
# mm.rename(columns={'question_en.1': 'question_en'}, inplace=True)
# mm['question_en'] = mm['question_en'].str.replace('\n', '', regex=False) 
# mm.to_csv(('./questions/processed/mmstar_translated_s3.csv'))

In [59]:
mm = sample_new_qs(df[~df['question_id'].isin(s1_2)], num_sample=100, types=['category', 'l2_category'], answer_type='choice', preprocess=True, filename=None) 

sampling new #100 from ['category', 'l2_category']
{
  "question_kr": "이미지의 텍스트 색상은 무엇입니까?",
  "options": "A: White 흰색, B: Blue 파란색, C: Black 검은색, D: Grey 회색"
}
{
  "question_kr": "testa는 어디에 있나요?",
  "options": "A: A 에이, B: C 씨, C: D 디, D: B 비"
}
{
  "question_kr": "이 그림에 보이는 물체는:",
  "options": "A: Is a colorless, flammable liquid that is commonly used as a solvent and fuel, B: Has a boiling point of 64.7°C, C: Can be toxic if ingested or absorbed through the skin, D: None of these options are correct. A: 용매 및 연료로 일반적으로 사용되는 무색의 인화성 액체이다, B: 끓는점이 64.7°C이다, C: 경구로 섭취하거나 피부를 통한 흡수로 독성이 있을 수 있다, D: 이들 옵션 중 어느 것도 옳지 않다."
}
{
  "question_kr": "이 그림에서 Dennis의 나이는 몇 살입니까?",
  "options": "A: 38, B: 45, C: 29, D: 47 A: 38, B: 45, C: 29, D: 47"
}
{
  "question_kr": "그 그림 속 액체에 단 하나의 ...가 포함되어 있다면",
  "options": "A: Ferric hydroxide. 철(III) 수산화물, B: Sodium hydroxide. 수산화나트륨, C: Sodium chloride. 염화나트륨, D: Copper sulfate. 황산구리(II)"
}
{
  "question_kr": "가장 가까운 침대에 있는 베개는 몇 개인가요?",
  "options": "A

In [14]:
mm = sample_new_qs(mm, num_sample=100, types=['category', 'l2_category'], answer_type='choice', preprocess=True, filename=None) 

,Unnamed: 0,qid,category,l2_category,qid.1,question_en,options,answer_type,question_kr
0,0,0,instance reasoning,single-instance reasoning,664,Where is the woman's blue bag located in the i...,"A: In her hand 그녀의 손에, B: On her shoulder 그녀의 ...",choice,이미지 속 여성의 파란 가방은 어디에 위치해 있나요?
1,1,1,coarse perception,image emotion,6,What feeling is shown in this image?\n,"A: engaged 참여하는, B: lonely 외로운, C: angry 화난, D...",choice,이 이미지에 어떤 감정이 나타나고 있나요?
2,2,2,instance reasoning,single-instance reasoning,662,Where is the rug in the living room located?\n,"A: Next to the door 문 옆에, B: Under the table 탁...",choice,거실의 러그는 어디에 위치해 있나요?
3,3,3,logical reasoning,common reasoning,773,Is there a teddy bear in the image?\n,"A: Can't tell 알 수 없음, B: No 아니오, C: Yes 네, D: ...",choice,이미지에 곰인형이 있나요?
4,4,4,instance reasoning,cross-instance relation reasoning,556,What is the relationship between the people in...,"A: commercial 상업적인, B: professional 전문적인, C: f...",choice,사진 속 사람들 간의 관계는 무엇입니까?
...,...,...,...,...,...,...,...,...,...
194,194,844,coarse perception,image scene and topic,92,What is the overall theme of the image?\n,"A: Beach vacation 해변 휴가, B: Athletic lifestyle...",choice,그 이미지를 전반적으로 나타내는 주제는 무엇입니까?
195,195,558,fine-grained perception,recognition,378,What is the color of the ears on the dessert i...,"A: Red, B: Brown, C: Black, D: White A: 빨간색, B...",choice,이미지의 오른쪽 아래에 위치한 디저트 아이템의 귀 색은 무엇입니까?
196,196,812,fine-grained perception,recognition,410,What is the main item on the counter in the im...,A: A cup of ice cream on a counter with a cone...,choice,사진 속 카운터에 있는 주요 아이템은 무엇입니까?
197,197,709,logical reasoning,common reasoning,814,Which corner doesn't have any food?\n,"A: top-right 오른쪽 위, B: top-left 왼쪽 위, C: botto...",choice,어떤 모서리에는 음식이 없나요?


In [61]:
mm = sample_new_qs(mm, num_sample=0, types=['category', 'l2_category'], answer_type='choice', preprocess=True, filename=None) 

{
  "question_kr": "이미지 속 여성의 파란 가방은 어디에 위치해 있나요?",
  "options": "A: In her hand 그녀의 손에, B: On her shoulder 그녀의 어깨에, C: On the ground 땅에, D: Inside the man's bag 그 남자의 가방 안에"
}
{
  "question_kr": "이 이미지에 어떤 감정이 나타나고 있나요?",
  "options": "A: engaged 참여하는, B: lonely 외로운, C: angry 화난, D: supportive 지지하는"
}
{
  "question_kr": "거실의 러그는 어디에 위치해 있나요?",
  "options": "A: Next to the door 문 옆에, B: Under the table 탁자 아래에, C: In front of the window 창문 앞에, D: Under the couch 소파 아래에"
}
{
  "question_kr": "이미지에 곰인형이 있나요?",
  "options": "A: Can't tell 알 수 없음, B: No 아니오, C: Yes 네, D: Maybe 아마도"
}
{
  "question_kr": "사진 속 사람들 간의 관계는 무엇입니까?",
  "options": "A: commercial 상업적인, B: professional 전문적인, C: friends 친구들, D: family 가족"
}
{
  "question_kr": "사진에서 아기가 어느 방향을 향하고 있나요?",
  "options": "A: left 왼쪽, B: right 오른쪽, C: up 위, D: down 아래"
}
{
  "question_kr": "다음에 무슨 일이 일어날까요?",
  "options": "A: the kid is gonna slide through 그 아이가 미끄러져 지나갈 거야, B: the kid is gonna crash into the other kid 그 아이가 다른 아이와 부딪힐 거야,

In [19]:
# not chosen questions MMSTAR 
new_mmstar = pd.read_excel(mmstar, sheet_name='others')  
sample_new_qs(new_mmstar, 200)

,Unnamed: 0,category,l2_category,question_id,question_en,question_kr,options,answer_type,question,answer,correct
0,0,instance reasoning,single-instance reasoning,664.0,Where is the woman's blue bag located in the i...,"이미지에서 여자의 파란 가방은 어디에 있나요? A: 그녀의 손에, B: 어깨에, C...","A: In her hand, B: On her shoulder, C: On the...",choice,NaN,NaN,NaN
1,1,coarse perception,image emotion,6.0,What feeling is shown in this image?\n,"이 이미지에는 어떤 느낌이 담겨 있나요? A: 참여, B: 외로운, C: 화난, D...","A: engaged, B: lonely, C: angry, D: supportive",choice,NaN,NaN,NaN
2,2,instance reasoning,single-instance reasoning,662.0,Where is the rug in the living room located?\n,"거실의 러그는 어디에 있나요?\nA: 문 옆, B: 테이블 아래, C: 창문 앞, ...","A: Next to the door, B: Under the table, C: I...",choice,NaN,NaN,NaN
3,3,logical reasoning,common reasoning,773.0,Is there a teddy bear in the image?\n,"이미지에 곰 인형이 있나요?\n옵션: A: 알 수 없습니다, B: 아니요, C: 네...","A: Can't tell, B: No, C: Yes, D: Maybe",choice,NaN,NaN,NaN
4,4,instance reasoning,cross-instance relation reasoning,556.0,What is the relationship between the people in...,"이미지 속 사람들 사이의 관계는 무엇인가요? A: 광고, B: 전문가, C: 친구,...","A: commercial, B: professional, C: friends, D...",choice,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
194,844,coarse perception,image scene and topic,92.0,What is the overall theme of the image?\n,NaN,"A: Beach vacation, B: Athletic lifestyle, C: ...",choice,What is the overall theme of the image?\n Opti...,C,0.153846
195,558,fine-grained perception,recognition,378.0,What is the color of the ears on the dessert i...,NaN,"A: Red, B: Brown, C: Black, D: White",choice,What is the color of the ears on the dessert i...,D,0.250000
196,812,fine-grained perception,recognition,410.0,What is the main item on the counter in the im...,NaN,A: A cup of ice cream on a counter with a con...,choice,What is the main item on the counter in the im...,B,0.166667
197,709,logical reasoning,common reasoning,814.0,Which corner doesn't have any food?\n,NaN,"A: top-right, B: top-left, C: bottom-left, D:...",choice,Which corner doesn't have any food?\n Options:...,D,0.181818


In [97]:
df = pd.concat(dfs)
df = df.dropna(axis=1)
df['question_id'] = pd.to_numeric(df['question_id'], errors='coerce')
# df['question_id']=df['question_id'].astype('int')
df['question_type'] = df['question_id'].map(question_type)
df[['question_en', 'question_kr']] = df['question'].str.split(
    pat='\n', 
    n=1,         # Only split on the FIRST newline found
    expand=True  # Expand the list into separate columns
)
# df['question_id'] = df['question_id'].t(int)
df.dropna(axis=0, inplace=True)
df['answer_type'] = 'text'
df.drop(columns=['question']).reset_index().to_csv('./questions/vqa_v1_text.csv') 

In [40]:
qids = df['question_id'].unique()
len(qids) 

278

In [ ]:
qtypes = df.groupby(['question_type']).count().sort_values(by=['question_id']).reset_index().question_type.unique()
qtype 

array(['is the person', 'has', 'was', 'how many people are',
       'what are the', 'what sport is', 'which', 'what is the man',
       'what kind of', 'is it', 'what is', 'what time', 'is the man',
       'can you', 'are the', 'do you', 'does this', 'do', 'is that a',
       'none of the above', 'what color', 'why', 'where is the', 'who is',
       'what color are the', 'what color is', 'what are', 'is he', 'is',
       'is there a', 'are they', 'are these', 'what is in the',
       'are there', 'does the', 'what is the', 'what type of', 'could',
       'are', 'how many', 'is there', 'are there any', 'what is on the',
       'what', 'what color is the', 'is this a', 'is this', 'is the'],
      dtype=object)